In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [3]:
!pip install datasets
!pip install transformers 

In [4]:
import torch
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased" #we will use bert base uncased for fine tune
tokenizer = AutoTokenizer.from_pretrained(checkpoint) #get the tokenizer

2024-05-28 06:44:51.254997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 06:44:51.255099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 06:44:51.377917: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [6]:
pd = raw_datasets['train'].to_pandas()
pd

,sentence,label,idx
0,hide new secretions from the parental units,0,0
1,"contains no wit , only labored gags",0,1
2,that loves its characters and communicates som...,1,2
3,remains utterly satisfied to remain the same t...,0,3
4,on the worst revenge-of-the-nerds clichés the ...,0,4
...,...,...,...
67344,a delightful comedy,1,67344
67345,"anguish , anger and frustration",0,67345
67346,"at achieving the modest , crowd-pleasing goals...",1,67346
67347,a patient viewer,1,67347


In [7]:
def tokenize(exemple):
   tokenized_sentence=tokenizer(exemple['sentence'],padding=True,truncation=True)
   return tokenized_sentence

In [8]:
tokinazed_dataset=raw_datasets.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#Data collators are objects that will
# form a batch by using a list of dataset elements as input.p

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [9]:
import pandas as pd

test = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

test.rename(columns={'review': 'sentence', 'sentiment': 'label'}, inplace=True)

test['label'] = test['label'].map({'negative': 0, 'positive': 1})

output_path = "/kaggle/working/modified_imdb_dataset.csv"
test.to_csv(output_path, index=False)

test.head()

,sentence,label
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(test)
tokenized_test = test_dataset.map(tokenize, batched=True)
test_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_test_df = tokenized_test.to_pandas()

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
#remove the columns "sentence" and "idx"
tokinazed_dataset=tokinazed_dataset.remove_columns(['sentence','idx'])
#rename the column "label" to "labels"
tokinazed_dataset=tokinazed_dataset.rename_column('label','labels')

In [12]:
tokinazed_dataset.set_format("torch") #comment this line if you on juypter

In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokinazed_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokinazed_dataset["validation"], batch_size=8, collate_fn=data_collator
)

In [14]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(tokinazed_dataset['train'],shuffle=True,batch_size=8,collate_fn=data_collator)
eval_dataloader=DataLoader(tokinazed_dataset['validation'],shuffle=True,batch_size=8,collate_fn=data_collator)

In [15]:
for batch in train_dataloader:
  break
for i,j in batch.items():
    print(f'{i}  :  {j.shape}')

labels  :  torch.Size([8])
input_ids  :  torch.Size([8, 66])
token_type_ids  :  torch.Size([8, 66])
attention_mask  :  torch.Size([8, 66])


In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,output_hidden_states=True)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
config=model.config
config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [18]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6519, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [19]:
hidden_states = outputs.hidden_states
#Bert has 12 hidden states
layer1=hidden_states[0]
layer1.shape

#explain the result torch.Size([8, N, 768])=>
# Explanation of the result:
#   - 8: This corresponds to the batch size, indicating that there are 8 sequences in the batch being processed.
#   - N: This represents the sequence length, indicating that each sequence has 59 tokens.
#   - 768: This denotes the hidden state length. For each token in each sequence, there are 768 hidden states produced by the mode

torch.Size([8, 66, 768])

In [20]:
#import the optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

25257


In [22]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [23]:
from tqdm.auto import tqdm
progress_bar=tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch={k:v.to(device) for k, v in batch.items()}
    outputs=model(**batch)
    loss=outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/25257 [00:00<?, ?it/s]

In [25]:
example_text = "This is an excellent movie"

inputs = tokenizer(example_text, return_tensors="pt")

inputs.to(device)

with torch.no_grad():
    outputs = model(**inputs)
outputs.logits

tensor([[-4.2499,  3.8006]], device='cuda:0')

In [26]:
predictions = torch.argmax(outputs.logits, dim=1)
print("Predicted class:", predictions.item())

Predicted class: 1


In [27]:
!pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00a 0:00:010m


In [28]:
import evaluate
metric=evaluate.load("glue","mrpc")
model.eval()
for batch in eval_dataloader:
  batch={k:v.to(device) for k,v in batch.items()}
  with torch.no_grad():
    outputs=model(**batch)
  logits=outputs.logits
  predictions=torch.argmax(logits,dim=-1)
  metric.add_batch(predictions=predictions,references=batch['labels'])
metric.compute()

{'accuracy': 0.9162844036697247, 'f1': 0.9175141242937852}